In [9]:
import pandas as pd
import requests
from pypdf import PdfReader
from io import BytesIO         
import tabula        
import re
import json
import extracao


In [5]:
df_pdfs_validos = pd.read_excel("../Base 5.1.xlsx", sheet_name="PDFs_validos")
df_pdfs_validos

,Nome,Núcleo,Trabalho,Data Publicação,Categoria,Categoria geal,Autores,PDF,Tipo de Arquivo/Link,Resumo
0,Sérgio Lazzarini,Metricis,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,2017,Artigo Acadêmico,Acadêmico,sergio giovanetti lazzarini,https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...,PDF,False
1,Leticia Faria de Carvalho Nunes,Saúde,https://repositorio.insper.edu.br/entities/pub...,2021,Working Paper,Pesquisa,leticia faria de carvalho nunes,https://repositorio.insper.edu.br/bitstreams/5...,text/html; charset=utf-8,True
2,Adriana Bruscato Bortoluzzo,Dados,https://repositorio.insper.edu.br/entities/pub...,2020,Artigo Acadêmico,Acadêmico,"adriana bruscato bortoluzzo, silva neto, anton...",https://repositorio.insper.edu.br/bitstreams/5...,text/html; charset=utf-8,True
3,Adriano Dutra Teixeira,Saúde,https://repositorio.insper.edu.br/entities/pub...,2023,Artigo Acadêmico,Acadêmico,Ok,https://repositorio.insper.edu.br/bitstreams/6...,text/html; charset=utf-8,True
4,Andrea Minardi,Metricis,https://repositorio.insper.edu.br/entities/pub...,2013,Working Paper,Pesquisa,ANDREA MARIA ACCIOLY FONSECA MINARDI; Rossi Jú...,https://repositorio.insper.edu.br/bitstreams/6...,text/html; charset=utf-8,True
...,...,...,...,...,...,...,...,...,...,...
517,Jefferson Oliveira de Moura,Pessoas do Poder Público,https://repositorio.insper.edu.br/entities/pub...,2024,Dissertação,Teses e Dissertações,Ok,https://repositorio.insper.edu.br/bitstreams/d...,text/html; charset=utf-8,True
518,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2020,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
519,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2023,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False
520,Henrique Yu Jiunn Wang,Pessoas do Poder Público,https://repositorio-api.insper.edu.br/server/a...,2022,Relatório de pesquisa,Pesquisa,NaN,https://repositorio-api.insper.edu.br/server/a...,PDF,False


In [6]:
df_pdfs_validos["PDF"]

0      https://www.scielo.br/j/rae/a/HCdY5w5zcp57GYkR...
1      https://repositorio.insper.edu.br/bitstreams/5...
2      https://repositorio.insper.edu.br/bitstreams/5...
3      https://repositorio.insper.edu.br/bitstreams/6...
4      https://repositorio.insper.edu.br/bitstreams/6...
                             ...                        
517    https://repositorio.insper.edu.br/bitstreams/d...
518    https://repositorio-api.insper.edu.br/server/a...
519    https://repositorio-api.insper.edu.br/server/a...
520    https://repositorio-api.insper.edu.br/server/a...
521    https://repositorio-api.insper.edu.br/server/a...
Name: PDF, Length: 522, dtype: object

In [11]:
lista_extraidos = []

# for url in df_pdfs_validos["PDF"]:
url = df_pdfs_validos["PDF"][2]
try:
    response = requests.get(url)
    response.raise_for_status() 

    pdf_file = BytesIO(response.content)
    reader = PdfReader(pdf_file)

    texto = ""
    if len(reader.pages) >= 10:
        for i in range(0,10):
            page = reader.pages[i]
            texto_formatado = "".join(page.extract_text())
            texto += texto_formatado
        resultado = extracao.extracao_langchain(texto)
        content = resultado.model_dump()
        data = json.dumps(content, ensure_ascii=False, indent=2, default=extracao.enum_encoder)
        lista_extraidos.append(data)
        
    else: 
        for i in range(len(reader.pages)):
            page = reader.pages[i]
            texto_formatado = "".join(page.extract_text())
            texto += texto_formatado
        resultado = extracao.extracao_langchain(texto)
        content = resultado.model_dump()
        data = json.dumps(content, ensure_ascii=False, indent=2, default=extracao.enum_encoder)
        lista_extraidos.append(data)

except:
    erro = {
"classificacao": None,
"metodologia": None,
"area_avaliada": None,
"ods_relacionada": None,
"ods_number": None,
"etapa": None,
"titulo": None,
"resumo": None
    }
    data = erro.model_dump()
    lista_extraidos.append(data)


lista_extraidos


['{\n  "classificacao": "Academica",\n  "metodologia": "Quantitativa",\n  "area_avaliada": "Desenvolvimento Social",\n  "ods_relacionada": "Trabalho decente e crescimento econômico",\n  "ods_number": 8,\n  "etapa": "Solução",\n  "titulo": "Diversification and Property Control Impact on the Performance of Brazilian REITs",\n  "resumo": "O estudo analisa a diversificação de tipos de propriedade e estratégias de controle de propriedade na performance dos FIIs brasileiros, utilizando o indicador Alpha de Jensen. A pesquisa abrange dados de painel de regressão dinâmica de 99 FIIs entre 2006 e 2016. O mercado de FIIs no Brasil, com capitalização de 20,2 bilhões de dólares em 2018, está em crescimento constante. Os dados foram coletados da plataforma Bloomberg, incluindo preço de fechamento, número de ações em circulação, valor patrimonial líquido e retorno bruto total dos FIIs. Os resultados indicam que a estratégia de controle de propriedade melhora a performance dos FIIs, sugerindo que evi